In [1]:

import pandas as pd
import numpy as np
#!conda install -c ulmo beautifulsoup4      #Uncomment this if BeautifulSoup is not installed on your system
#!conda install -c ulmo urllib3             #Uncomment this if urllib3 is not installed on your system
from bs4 import BeautifulSoup
import urllib3
# To scrap the data
CONST_wikipediaLink = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

# Key for the dataframe we will use
CONST_dfColumns = ['PostalCode', 'Borough', 'Neighborhood']

# Size of the radius to retrieve venues from FoursquareAPI, and limit of venues per neighbourhood
CONST_venuesRadiusScan = 1100
CONST_venuesLimit = 100

In [2]:
http = urllib3.PoolManager()
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Scrape the raw xml data from Wikipedia
response = http.request('GET', CONST_wikipediaLink)
soup = BeautifulSoup(response.data, "lxml")

#soup

df_neighbourhoods = pd.DataFrame(columns=CONST_dfColumns)
fullList = []
# Retrieve all neighbourhoods, in the <tr> tags
neighbourhoodsList = soup.body.table.find_all('tr')

# Loop through each neighbourhood, in the <td> tags
for neighbourhood in neighbourhoodsList:
    neighbourElts = neighbourhood.find_all('td')
    
    # Loop through each attritube of the current neighbourhood : name, title, and wikipedia url
    tmpList = []
    for elt in neighbourElts:
        # Remove the tags & newlines
        tmpList.append(str(elt.get_text().strip()))
        
    fullList.append(tmpList)

# Add the scraped nneighbourhoods into the dataframe
df_neighbourhoods=pd.DataFrame(fullList,columns=CONST_dfColumns)
df_neighbourhoods.head()

,PostalCode,Borough,Neighborhood
0,None,None,None
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


In [3]:
# Filter all unassigned neighbourhoods
df_neighbourhoods = df_neighbourhoods[(df_neighbourhoods.Borough.notnull())]
df_neighbourhoods = df_neighbourhoods[(df_neighbourhoods.Borough != "Not assigned")]

# Group by PostalCode/Borough
df_neighbourhoods = pd.DataFrame(df_neighbourhoods.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(list)).reset_index()
df_neighbourhoods['Neighborhood'] = df_neighbourhoods['Neighborhood'].apply(lambda x: ', '.join(x))

# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough
df_neighbourhoods.loc[ (df_neighbourhoods.Neighborhood.isnull() == True) |
               (df_neighbourhoods.Neighborhood == "Not assigned")
               , 'Neighborhood'] = df_neighbourhoods.Borough

df_neighbourhoods.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [4]:
df_neighbourhoods.shape

(103, 3)

### Part 2 - Add the latitude and the longitude coordinates of each neighbourhood

In [5]:
#!conda install -c conda-forge geocoder             #Uncomment this if geocoder is not installed on your system
import geocoder # import geocoder

ModuleNotFoundError: No module named 'geocoder'

In [ ]:

# This function returns the latitude and longitude of the given postal code, in Toronto
def getCoordsByPostalCode(postal_code):
    # initialize to None : this variable will allow us to loop until geocoder responds with the coordinates
    lat_lng_coords = None

    # loop until we get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng

    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    
    return latitude, longitude

postalCodesWithCoordsList = []

# Loop through each postal code from our dataframe
for postalCode in df_neighbourhoods['PostalCode']:
    # Fill the temp list with the coordinates from geocoder
    latitude, longitude = getCoordsByPostalCode(postalCode)
    postalCodesWithCoordsList.append([postalCode, latitude, longitude])
    
# Transform the temp list into a dataframe
df_coords = pd.DataFrame(postalCodesWithCoordsList)
df_coords.columns = ['PostalCode', 'Latitude', 'Longitude']

# Merge the coordinates dataframe with the original neighbourhoods dataframe (key : postal code)
df_neighbourhoods = pd.merge(df_neighbourhoods, df_coords, on='PostalCode')

df_neighbourhoods_toronto = df_neighbourhoods

df_neighbourhoods.head()

### Part 3 - Neighbourhoods clustering using venues from FoursquareAPI

In [6]:
#!conda install -c conda-forge folium                #Uncomment this if folium is not installed on your system
import folium
import requests

# import k-means from clustering stage
from sklearn.cluster import KMeans

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [ ]:

# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude_toronto, longitude_toronto], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood, postalCode in zip(df_neighbourhoods_toronto['Latitude'], df_neighbourhoods_toronto['Longitude'], df_neighbourhoods_toronto['Borough'], df_neighbourhoods_toronto['Neighborhood'], df_neighbourhoods_toronto['PostalCode']):
    label = '{}'.format(postalCode)
    label = folium.Popup(label, parse_html=True)

    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto) 
    
map_torontoaddress = 'Toronto, ON'

# initialize to None
lat_lng_coords = None

# loop until we get the coordinates
while(lat_lng_coords is None):
    g = geocoder.arcgis('Toronto, Ontario')
    lat_lng_coords = g.latlng

latitude_toronto = lat_lng_coords[0]
longitude_toronto = lat_lng_coords[1]

print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude_toronto, longitude_toronto))

In [ ]:

# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude_toronto, longitude_toronto], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood, postalCode in zip(df_neighbourhoods_toronto['Latitude'], df_neighbourhoods_toronto['Longitude'], df_neighbourhoods_toronto['Borough'], df_neighbourhoods_toronto['Neighborhood'], df_neighbourhoods_toronto['PostalCode']):
    label = '{}'.format(postalCode)
    label = folium.Popup(label, parse_html=True)

    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto) 
    
map_toronto

In [ ]:
# create map of Toronto using latitude and longitude values
map_toronto_radius = folium.Map(location=[latitude_toronto, longitude_toronto], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood, postalCode in zip(df_neighbourhoods_toronto['Latitude'], df_neighbourhoods_toronto['Longitude'], df_neighbourhoods_toronto['Borough'], df_neighbourhoods_toronto['Neighborhood'], df_neighbourhoods_toronto['PostalCode']):
    label = '{}'.format(postalCode)
    label = folium.Popup(label, parse_html=True)

    folium.Circle(
        [lat, lng],
        radius=CONST_venuesRadiusScan,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.3).add_to(map_toronto_radius) 
    
map_toronto_radius

##Utilizing the Foursquare API to explore and segment neighborhoods

CLIENT_ID = 2GFKIZFPHZMTSMVADOZIQK2SHKOMAFGOBFL03UPJAFI1VU4C # your Foursquare ID
CLIENT_SECRET = HTHH43W3AKEFZH5XDJKZC0YXMQ0DH34HB5CJPT0IJZLEAIWF # your Foursquare Secret
VERSION = '20180604'   #'20180605' Foursquare API Version
LIMIT = 30

In [ ]:

def getNearbyVenues(postalCodes, boroughs, neighbourhoods, latitudes, longitudes):
    
    venues_list=[]
    # Loop through each neighbourhood given in parameters
    for postalCode, borough, neighbourhood, lat, lng in zip(postalCodes, boroughs, neighbourhoods, latitudes, longitudes):
            
        # create the API request URL to explore the neighbourhood using FoursquareAPI
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            CONST_venuesRadiusScan, 
            CONST_venuesLimit)

        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue : name, latitude, longitude, and the categories' names
        venues_list.append([(
            postalCode,
            borough,
            neighbourhood, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    # add the venues in the dataframe
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = [
                        'PostalCode',
                        'Borough',
                        'Neighborhood', 
                        'Neighborhood Latitude', 
                        'Neighborhood Longitude', 
                        'Venue', 
                        'Venue Latitude', 
                        'Venue Longitude', 
                        'Venue Category'
    ]
    
    return(nearby_venues)

In [ ]:
# Get the venues for each neighbourhood
toronto_venues = getNearbyVenues(  
                                    postalCodes=df_neighbourhoods_toronto['PostalCode'],
                                    boroughs=df_neighbourhoods_toronto['Borough'],
                                    neighbourhoods=df_neighbourhoods_toronto['Neighborhood'],
                                    latitudes=df_neighbourhoods_toronto['Latitude'],
                                    longitudes=df_neighbourhoods_toronto['Longitude']
                                  )

toronto_venues.head()

##Analyse each neighborhood : each type of venue will be one hot encoded so we will be able to perform a K-means clustering on the dataframe

In [ ]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add postalCode, borough, and neighborhood column back to dataframe
toronto_onehot['PostalCode'] = toronto_venues['PostalCode'] 
toronto_onehot['Borough'] = toronto_venues['Borough'] 

toronto_onehot.pop('Neighborhood')
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move postalCode, borough, neighborhood column to the first column
for i in range(0, 3):
    toronto_onehot = toronto_onehot[[toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])]

toronto_onehot.head()

In [ ]:
toronto_grouped = toronto_onehot.groupby(['PostalCode','Borough', 'Neighborhood']).mean().reset_index()
toronto_grouped.head()

In [ ]:
num_top_venues = 5

# Iterate through all the grouped dataframe
for index, row in toronto_grouped.iterrows():
    tempPostalCode = row['PostalCode']
    tempBorough = row['Borough']
    tempNeighborhood = row['Neighborhood']
    
    print("----"+tempPostalCode + " / " + tempBorough + " / " + tempNeighborhood +"----")
    
    # Create a temp df filtered on the current neighbourhood (key : postal code x borouhg x neighbourhood)
    temp = toronto_grouped[
        (toronto_grouped.PostalCode == tempPostalCode) &
        (toronto_grouped.Borough == tempBorough) &
        (toronto_grouped.Neighborhood == tempNeighborhood)
    ].T.reset_index()
    
    temp.columns = ['venue','freq']

    # We skip the key PostalCode x Borough x Neighbourhood : length = 3, iloc[3:]
    temp = temp.iloc[len(CONST_dfColumns):]
    temp['freq'] = temp['freq'].astype(float)
    
    # Round the frequency with two digits
    temp = temp.round({'freq': 2})
    
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

In [ ]:

def return_most_common_venues(row, num_top_venues):
    # Remove the key PostalCode x Borough x Neighbourhood from the row
    row_categories = row.iloc[len(CONST_dfColumns):]
    
    # Sort ascending
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    # Return the top num_top_venues
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['PostalCode', 'Borough', 'Neighborhood']

for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe, and set it with the columns names
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)

# add the keys from the grouped dataframe (Postal code x Borough x Neighborhood)
neighborhoods_venues_sorted['PostalCode'] = toronto_grouped['PostalCode']
neighborhoods_venues_sorted['Borough'] = toronto_grouped['Borough']
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

# loop through each rows
for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, len(CONST_dfColumns):] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

In [ ]:

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)
toronto_grouped_clustering = toronto_grouped_clustering.drop('PostalCode', 1)
toronto_grouped_clustering = toronto_grouped_clustering.drop('Borough', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

In [ ]:

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [ ]:

toronto_merged = df_neighbourhoods_toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index(['PostalCode','Borough', 'Neighborhood']), on=['PostalCode','Borough', 'Neighborhood'])

toronto_merged.head() # check the last columns!

In [ ]:

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, postalCode, borough, neighborhood, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['PostalCode'], toronto_merged['Borough'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(postalCode) + ' - Cluster ' + str(cluster), parse_html=True)
    cluster = int(cluster)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [ ]:
### Examine clusters

Cluster 0 focuses on Parks, Gyms, and places to practice sports and/or chillout outside.
We can see that most of the neighbourhoods categorised in this cluster are near several parks on the map.

In [ ]:

toronto_merged[toronto_merged['Cluster Labels'] == 0].head(10)

In [ ]:
### Cluster 1 focuses on the diversity of restaurants, coffee shops, and bars.
We can see that the downtown of Toronto is highly categorised in this cluster.

In [ ]:

toronto_merged[toronto_merged['Cluster Labels'] == 1].head(10)

In [ ]:
### Cluster 2 is an outlier, its 1st common Venue is Zoo Exhibit, it also features Hobby shops, Electronics Stores ...

In [ ]:
toronto_merged[toronto_merged['Cluster Labels'] == 2].head(10)

In [ ]:
###Cluster 3 is an outlier, its 1st commont venue is Bakery, and it also features Farms, Event spaces ...

In [ ]:

toronto_merged[toronto_merged['Cluster Labels'] == 3].head(10)

In [ ]:
### Cluster 4 focuses on Fast foods, pizzas and restaurants in general.
We can see that this cluster is predominant outside of the downtown.

In [ ]:
toronto_merged[toronto_merged['Cluster Labels'] == 4].head(10)